The tweets in Founta et al. have been classified using the model from Blodgett et al. This assigns each tweet a vector of probabilities corresponding to four different racialized language models. To use this information in the STM I need to convert these probabilities into discrete values. 

Note that Blodgett et al. warn against using the other and asian categories, so for the topic model I should throw out those not majority white or African-American.

In [18]:
import pandas as pd

In [19]:
df = pd.read_csv('data/founta_race_annotated.csv')

In [20]:
df.shape

(91341, 7)

In [21]:
df.head()


,id,text,label,afam,hisp,asian,white
0,0,Beats by Dr. Dre urBeats Wired In-Ear Headphon...,spam,0.379062,0.222205,0.193619,0.205114
1,1,RT @Papapishu: Man it would fucking rule if we...,abusive,0.187467,0.187928,0.118104,0.506501
2,2,It is time to draw close to Him &#128591;&#127...,neither,0.223829,0.480416,0.013496,0.282260
3,3,if you notice me start to act different or dis...,neither,0.466657,0.331978,0.007351,0.194014
4,4,"Forget unfollowers, I believe in growing. 7 ne...",neither,0.106735,0.186906,0.089628,0.616732


Note some columns have values of -9. This indicates that no remaining words were present after tokenization using the Blodgett algorithm. These should be dropped from the analysis.

In [49]:
def getHighestProbAndThreshold(df, threshold=.6):
    race_cats = ['afam', 'hisp', 'asian', 'white']
    max_race = []
    max_vals = []
    threshold_met = []
    for _, r in df.iterrows():
        values = [float(r['afam']), float(r['hisp']), float(r['asian']), float(r['white'])]
        max_val = max(values)
        max_idx = values.index(max_val)
        max_race.append(race_cats[max_idx])
        max_vals.append(max_val)
        if max_val >= threshold:
            threshold_met.append(True)
        else:
            threshold_met.append(False)
    df['max_race'] = max_race
    df['max_vals'] = max_vals
    df['threshold_met'] = threshold_met
    return(df)

In [50]:
df = getHighestProbAndThreshold(df)

In [51]:
df.tail()

,id,text,label,afam,hisp,asian,white,max_race,max_vals,threshold_met
91336,99991,RT @shangros: my fucking queen https://t.co/wa...,abusive,0.216189,0.305741,0.289655,0.188415,hisp,0.305741,False
91337,99992,#Osteporosis treated with #PEMF - rebuild bone...,neither,0.218286,0.215283,0.183034,0.383396,white,0.383396,False
91338,99993,@LGUSAMobile why does my phone screen keeps fl...,neither,0.287190,0.459262,0.008520,0.245029,hisp,0.459262,False
91339,99994,#bigdata vs. #reality ... but equally applies ...,neither,0.098390,0.141443,0.219687,0.540480,white,0.540480,False
91340,99995,"you can do whatever you choose, if you first g...",neither,0.082188,0.306929,0.003522,0.607361,white,0.607361,True


In [57]:
from collections import Counter
Counter(df.query('label != "spam" and afam != -9')['max_race'])

Counter({'white': 52143, 'hisp': 9584, 'afam': 8583, 'asian': 6839})

In [53]:
df.shape

(91341, 10)

In [54]:
df.query('threshold_met').shape

(22967, 10)

In [58]:
df.query('afam == -9').shape

(647, 10)

In [59]:
df.to_csv('data/founta_race_annotated_2.csv')